# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## W&B Setup
🪄 Install `wandb` library and login

Start by installing the library and logging in to your free account.

In [1]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.2/300.2 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Installing libraries 📚

In [ ]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5

## Setting things up for the environment 🌍 🪖

In [ ]:
import os

#saving current directory just to be sure
content_dir = os.getcwd()

#cloning Fufi repo from git
!git clone https://github.com/Gaianeve/gym-Fufi.git
#installing things
!pip install /content/gym-Fufi

In [ ]:
# Enter the environment directory
%cd /content/gym-Fufi
# Actually importing the library for our environment
import gym_Fufi

In [ ]:
#get back to content directory so I save everything there
%cd ..
!pwd

## importing libraries and functions 📚


In [ ]:
#libraries
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## Log needed function from files 📡
Loading files directly from git, so I don't have to upload it by hand.

In [ ]:
#get files from git
!git clone https://github.com/Gaianeve/FUFONE.git

In [ ]:
!pwd
%cd FUFONE/PPO
from environment import vectorize_env
from agent_class import Agent
from agent_utils import anneal, collect_data, GAE, PPO_train_agent, evaluate_agent

#back to main directory
%cd ..
#import main function from file
from main_function_sweep import main, parse_args

In [ ]:
#back to content directory
%cd ..

## Define the sweep 📑
I assume to start with a random search, that is why we take random distribution:
*   `ent_coef ` ➡ Discrete uniform distribution on integers. Between 0.01, that is the default value and 1, that is the thing that works best for now.
*  ` num_envs`  ➡ list of values to try out
*   `learning_rate` ➡  Quantized log uniform. Returns `round(X / q) * q` where `X` is` log_uniform_values`. Basically, a pretty good approximation of a log_q uniform distribution


In [ ]:
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_Fufi",
    "metric": {"goal": "maximize", "name": "sum_episodes"},
    "parameters": {
       "num_steps": {"values": [128, 256, 512, 2048, 4096, 8192]},
       "ent_coef": {'distribution': 'uniform', 'min': 0.01,'max': 1},
       "num_envs" : {"values": [2,4,8,16,32]},
       "learning_rate":  {'distribution': 'uniform', 'min': 1.5e-7,'max': 1.5e-4}
    },
}


In [ ]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [ ]:
import wandb
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="Fufi_sweep_trials")  # Set up the sweep

    def sweep_main():
       with wandb.init() as run:
        # retrieve the parser
        args = parse_args()

        # Update args with sweep parameters
        args.learning_rate = wandb.config.learning_rate
        args.num_steps = wandb.config.num_steps
        args.ent_coef = wandb.config.ent_coef
        args.num_envs = wandb.config.num_envs

        main()


In [ ]:
 wandb.agent(sweep_id, function=sweep_main, count = 10)

In [14]:
import shutil
shutil.rmtree('FUFONE')